In [69]:
import pandas as pd
import numpy as np
import google.generativeai as genai
import os
from dotenv import load_dotenv

# load dot env
env_file_name = "/home/wildanaziz/dtp-data-pipeline/gemini.env"

#fetch api gemini
if load_dotenv(dotenv_path=env_file_name):
    print(f"File{env_file_name} berhasil dimuat..")
else:
    print(f"File {env_file_name} tidak berhasil dimuat..")
    
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    raise ValueError(f"API KEY gagal dimuat pada file {env_file_name}..")
    
    
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-flash-lite')
print(f"konfig berhasil")

File/home/wildanaziz/dtp-data-pipeline/gemini.env berhasil dimuat..
konfig berhasil


In [70]:
try:
    df_lowongan = pd.read_csv("/home/wildanaziz/dtp-data-pipeline/data_raw/Data Lowongan Pekerjaan 6001-9000 - Sheet1.csv")
    df_pon_tik = pd.read_csv("/home/wildanaziz/dtp-data-pipeline/data_raw/PON TIK FIX - Sheet1.csv")
except FileNotFoundError as e:
    print(f"Error: {e}. Pastikan file CSV berada di direktori yang sama.")
    exit()

missing_values = ["Tidak Ada", "Tidak ada", "-", "", "Not Applicable"]

df_lowongan.replace(missing_values, np.nan, inplace=True)
print("Jumlah missing values awal di beberapa kolom:")
print(df_lowongan[['Level Pekerjaan', 'Industri', 'Spesial Info', 'Skillset', 'Tools']].isnull().sum())

Jumlah missing values awal di beberapa kolom:
Level Pekerjaan    921
Industri           330
Spesial Info        32
Skillset             3
Tools              337
dtype: int64


In [71]:
okupasi_to_level = df_pon_tik.set_index('OKUPASI')['LEVEL'].to_dict()

def impute_level_pekerjaan(row):
    if pd.notna(row['Level Pekerjaan']):
        return row['Level Pekerjaan']

    if pd.isna(row['Okupasi']):
        return None
        
    # clean okupasi row
    okupasi_lowongan = str(row['Okupasi'])
    
    okupasi_upper = okupasi_lowongan.upper()
    
    pos_separate = okupasi_upper.find(' - LEVEL')
    
    if pos_separate != -1:
        okupasi = okupasi_upper[:pos_separate].strip()
    else:
        okupasi = okupasi_upper.strip()

    for okupasi_pon_tik, level in okupasi_to_level.items():
        if okupasi in str(okupasi_pon_tik).upper():
            return level


    return None

df_lowongan['Level Pekerjaan'] = df_lowongan.apply(impute_level_pekerjaan, axis=1)

print("\nImputasi selesai")


Imputasi selesai


In [72]:
#pemetaan
def kategorisasi_level(level):
    try:
        level = int(float(level))
        if level <= 2:
            return 'Internship/Magang/OJT'
        elif level <= 4:
            return 'Lulusan Baru/Junior/Entry Level/Fresh Graduate'
        elif level == 5:
            return 'Associate'
        elif level == 6:
            return 'Mid Senior Level'
        elif level == 7:
            return 'Supervisor/Asisten Manager'
        elif level >= 8:
            return 'Direktur/Eksekutif'
        else:
            return np.nan
    except (ValueError, TypeError):
        return level

# categorized
df_lowongan['Level Pekerjaan'] = df_lowongan['Level Pekerjaan'].apply(kategorisasi_level)

print("Kategorisasi 'Level Pekerjaan' selesai.")
print("\nContoh hasil setelah perbaikan:")
print(df_lowongan[['Okupasi', 'Level Pekerjaan']].head())

Kategorisasi 'Level Pekerjaan' selesai.

Contoh hasil setelah perbaikan:
                                   Okupasi  \
0               BUSINESS ANALYST - Level 6   
1  ASSISTANT COMPUTER TECHNICIAN - Level 4   
2                   DATA ANALYST - Level 6   
3                SYSTEM ENGINEER - Level 7   
4  ASSISTANT COMPUTER TECHNICIAN - Level 4   

                                  Level Pekerjaan  
0                                Mid Senior Level  
1  Lulusan Baru/Junior/Entry Level/Fresh Graduate  
2                                       Associate  
3                      Supervisor/Asisten Manager  
4  Lulusan Baru/Junior/Entry Level/Fresh Graduate  


In [75]:
df_lowongan.head(100)

,No,Pekerjaan,Level Pekerjaan,Status Pekerjaan,Industri,Spesial Info,Deskripsi Pekerjaan,Jenis_Okupasi,Okupasi,Area Fungsi,Skillset,Tools
0,6001,Business Analyst,Mid Senior Level,Full time,NaN,Analis Bisnis/Sistem (Teknologi Informasi & Ko...,Analisis Kebutuhan | Berinteraksi dengan peman...,PON-TIK,BUSINESS ANALYST - Level 6,Pengembangan Produk Digital,analytical skills;communication skills;busines...,NaN
1,6002,IT Support,Lulusan Baru/Junior/Entry Level/Fresh Graduate,Full time,Hospitality & Tourism,Meja Bantuan & Dukungan IT (Teknologi Informas...,"PT. Hatten Bali Tbk., sebuah perusahaan yang b...",PON-TIK,ASSISTANT COMPUTER TECHNICIAN - Level 4,Layanan Teknologi Informasi,technical support;diagnosis;troubleshooting;pr...,windows
2,6003,Operational Data Analyst,Associate,Full time,NaN,Analis Bisnis/Sistem (Teknologi Informasi & Ko...,KUALIFIKASI : | Pendidikan minimal S1 dari jur...,PON-TIK,DATA ANALYST - Level 6,Sains Data-Kecerdasan Artifisial,"pengolahan dan analisis data operasional, root...","Excel tingkat lanjut, Google Sheets, Google Da..."
3,6004,IT Infrastructure Tester,Supervisor/Asisten Manager,Kontrak/Temporer,NaN,Uji Coba & Penjaminan Mutu (Teknologi Informas...,Tanggung Jawab: | Melakukan pengujian pada inf...,PON-TIK,SYSTEM ENGINEER - Level 7,Teknologi Dan Infrastruktur,infrastructure testing;test planning;test case...,jmeter;loadrunner;network monitoring tools;ser...
4,6005,IT Support (Purwokerto),Lulusan Baru/Junior/Entry Level/Fresh Graduate,Full time,FMCG Manufacturing,Meja Bantuan & Dukungan IT (Teknologi Informas...,Deskripsi Pekerjaan: | Sebagai IT Support Anda...,PON-TIK,ASSISTANT COMPUTER TECHNICIAN - Level 4,Layanan Teknologi Informasi,installation;evaluation;improvement;network sy...,microsoft;windows;linux;mikrotik
...,...,...,...,...,...,...,...,...,...,...,...,...
95,6096,Developer Back End,Mid Senior Level,Full time,NaN,Developer/Programmer (Teknologi Informasi & Ko...,"Job Descriptions: | Design, develop, and maint...",PON-TIK,BACKEND DEVELOPER - Level 6,Pengembangan Produk Digital,api design;backend development;microservices a...,node.js;restful apis;microservices;postgresql;...
96,6097,Applicant Engineer,Associate,Kontrak/Temporer,NaN,Lainnya (Teknik),Job Desc: | Review completeness of received in...,PON-TIK,ASSOCIATE APPLICATION ENGINEER - Level 5,Teknologi Dan Infrastruktur,technical analysis;commercial analysis;proposa...,NaN
97,6098,Junior Data Center - Banking Industry,Associate,Kontrak/Temporer,Consulting Services,Administrasi Jaringan & Sistem (Teknologi Info...,"• Kandidat memahami cara kerja, operasional da...",PON-TIK,ASSOCIATE DATA CENTER TECHNICIAN - Level 5,Teknologi Dan Infrastruktur,data center operations;infrastructure manageme...,genset;ups;pac;fire system;access door lock;su...
98,6099,IT Security (Blue Team),Mid Senior Level,Full time,Consulting Services,Keamanan (Teknologi Informasi & Komunikasi),Location: | Placement in one of our clients in...,PON-TIK,CYBERSECURITY ANALYST - Level 6,Keamanan Informasi Dan Siber,security operations;incident response;threat a...,fortigate firewall;sophos;crowdstrike;forti em...


In [76]:
import time
import numpy as np
import pandas as pd

def impute_with_gemini_final(row, column_to_impute):
    if pd.notna(row[column_to_impute]):
        return row[column_to_impute]

    pekerjaan = row['Pekerjaan']
    deskripsi = row['Deskripsi Pekerjaan']

    if pd.isna(pekerjaan) or pd.isna(deskripsi):
        return np.nan
    
    # buat prompt
    prompts = {
        'Industri': f"Berdasarkan pekerjaan '{pekerjaan}' dan deskripsi '{deskripsi}', apa nama industri yang paling sesuai? Berikan satu jawaban singkat saja. Contoh: Teknologi Informasi.",
        'Spesial Info': f"Dari deskripsi pekerjaan '{deskripsi}' untuk posisi '{pekerjaan}', identifikasi 1-2 kualifikasi khusus yang paling menonjol. Jika tidak ada, tulis 'Tidak ada'. Berikan jawaban singkat.",
        'Skillset': f"Berdasarkan deskripsi '{deskripsi}' untuk posisi '{pekerjaan}', sebutkan 5 skill utama yang dibutuhkan, pisahkan dengan titik koma.",
        'Tools': f"Berdasarkan deskripsi '{deskripsi}' untuk posisi '{pekerjaan}', sebutkan 3 tools/software utama yang digunakan, pisahkan dengan titik koma."
    }
    
    prompt = prompts.get(column_to_impute)
    if not prompt:
        return np.nan

    try:
        response = model.generate_content(prompt)
        
        
        time.sleep(4) 
        
        return response.text.strip()
    except Exception as e:
        print(f"Error pada baris {row.name} untuk kolom '{column_to_impute}': {e}")
        return np.nan

In [ ]:
columns_to_impute_gemini = ['Industri', 'Spesial Info', 'Skillset', 'Tools']

for col in columns_to_impute_gemini:
    print(f"\nMemulai imputasi penuh untuk kolom: {col} (ini akan memakan waktu)...")
    df_lowongan[col] = df_lowongan.apply(
        lambda row: impute_with_gemini_final(row, col) if pd.isna(row[col]) else row[col],
        axis=1
    )
    print(f"Imputasi untuk kolom '{col}' selesai.")

print("\nProses imputasi menggunakan Gemini telah selesai sepenuhnya.")


Memulai imputasi penuh untuk kolom: Industri (ini akan memakan waktu)...


E0000 00:00:1759672841.426164  247748 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
